In [ ]:
%defaultDatasource jdbc:h2:mem:db

In [ ]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [1]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

java.lang.RuntimeException:  No datasource

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [ ]:
CREATE VIEW InteracaoParMedicamento AS SELECT dua.codeDrug codeDrugA, dub.codeDrug codeDrugB
FROM (SELECT * FROM DrugUse ORDER BY codeDrug) AS dua CROSS JOIN (SELECT * FROM DrugUse ORDER BY codeDrug) AS dub
WHERE dua.idPerson = dub.idPerson AND dua.codeDrug <> dub.codeDrug AND dua.codeDrug < dub.codeDrug

CREATE VIEW GrupoMais30 AS SELECT ipm.codeDrugA drugA, ipm.codeDrugB drugB, COUNT(*) AS weight
FROM InteracaoParMedicamento ipm
GROUP BY codeDrugA, codeDrugB
HAVING ocorrencia > 30

SELECT * FROM GrupoMais30

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [ ]:
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;

In [ ]:
CREATE VIEW GrupoMais30x AS SELECT di.druga drugA, di.drugb drugB, di.weight
FROM DrugInteraction di
WHERE di.weight > 30

SELECT * FROM GrupoMais30x

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [ ]:
SELECT gm30.drugA, gm30.drugB, (CAST(gm30.weight AS DOUBLE)/(SELECT SUM(weight) FROM GrupoMais30x)) AS suporte
FROM GrupoMais30x gm30
ORDER BY suporte

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [ ]:
CREATE VIEW AparicaoDrugA AS SELECT drugA, SUM(weight) AS total
FROM GrupoMais30x 
GROUP BY drugA

SELECT gm30.drugA, gm30.drugB, (CAST(gm30.weight AS DOUBLE)/ada.total) AS confianca
FROM GrupoMais30x gm30, AparicaoDrugA ada
WHERE gm30.drugA = ada.drugA
ORDER BY confianca

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [ ]:
CREATE VIEW DrogasEC AS SELECT du.codeDrug, du.idPerson
FROM DrugUse du, SideEffect se
WHERE du.idPerson = se.idPerson

CREATE VIEW Pessoa1DrogaEC AS SELECT idPerson, COUNT(*) AS total
FROM DrogasEC
GROUP BY idPerson
HAVING total = 1

CREATE VIEW PessoaMultDrogas AS SELECT idPerson, COUNT(*) AS total
FROM DrogasEC
GROUP BY idPerson
HAVING total > 1

CREATE VIEW DrogaECSozinha AS SELECT def.codeDrug Drug, COUNT(*) AS total
FROM DrogasEC def, Pessoa1Droga p1d
WHERE def.idPerson = p1d.idPerson
GROUP BY def.codeDrug

CREATE VIEW ParDrogaEC AS SELECT deca.codeDrug drugA, decb.codeDrug drugB, COUNT(*) AS total
FROM PessoaMultDrogas pmd, (SELECT * FROM DrogasEC ORDER BY codeDrug) AS deca CROSS JOIN (SELECT * FROM DrogasComEC ORDER BY codeDrug) AS decb
WHERE deca.idPerson = pmd.idPerson AND deca.idPerson = decb.idPerson AND deca.codeDrug <> decb.codeDrug AND deca.codeDrug < decb.codeDrug
GROUP BY drugA, drugB

SELECT pdec.drugA, pdec.drugB, (CAST(pdec.total AS DOUBLE)/(SELECT defs.total FROM DrogaECsozinha defs, ParDrogaEC pd WHERE defs.Drug = pd.DrugA)) AS relacaoDrogaASozinha
FROM ParDrogaEC
ORDER BY pdec.drugA